### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [61]:
baby_df['review']=baby_df['review'].astype(str)

In [62]:
#b)
baby_df["review"].replace(np.nan, "", inplace=True)
#short test:
baby_df["review"][38] == baby_df["review"][38]

True

Initially, in the process of eliminating all NaN data, we employ a function called 'replace' to substitute one sentence with another. In this function, we replace any instances of NaN with an empty space.

In [63]:
#a)
baby_df["review"] = baby_df["review"].apply(remove_punctuation)

#short test:
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

Subsequently, following the conversion of all reviews to text format, we utilize a provided function that eliminates all punctuation marks. This is accomplished by employing the ‘apply’ option provided by the pandas package.

In [36]:
#c)
a = 0
for x in baby_df["rating"]:

    if x == 3:
        baby_df.drop(index = a, inplace = True, axis=0)
        a=a+1
    else:
        a=a+1
        continue

#short test:
sum(baby_df["rating"] == 3)

0

Moving forward, the subsequent task involves eliminating the lines with a rating of 3. This is achieved by searching through the file for these specific lines and then removing them using the standard for loop in conjunction with the drop function.

In [37]:
#d) 
baby_df["rating"].replace(1,-1, inplace = True)
baby_df["rating"].replace(4,1, inplace = True)
baby_df["rating"].replace(5,1, inplace = True)
baby_df["rating"].replace(2,-1, inplace = True)
#short test:
sum(baby_df["rating"]**2 != 1)

0

Upon completing this task, the final step entails converting all scores above 3 to 1 and those below 3 to -1. To accomplish this, we utilize the 'replace' function as employed previously, focusing on substituting numerical values this time.

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

# print(vectorizer.get_feature_names())
print(X_train_example.todense())



[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [39]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [40]:
#a)
from sklearn.model_selection import train_test_split
train, test = train_test_split(baby_df, train_size=0.8, test_size=0.2)

In [41]:
#b)
vectorizer = CountVectorizer()

x = vectorizer.fit_transform(list(train["review"]))
y = train["rating"]
x_test = vectorizer.transform(list(test["review"]))
y_test = test["rating"]



In the subsequent task, we partition the data into training and test sets. Following that, we convert the words present in the reviews into arrays of numbers (vectors). This process enables us to instruct the program in recognizing specific characteristics of words through numerical representations.

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [42]:
#a)
model = LogisticRegression(max_iter=200)
model.fit(x, y)

C:\Users\User\PycharmProjects\Data-Science\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200)

In [43]:
#b)
zipped_coef = list(zip(list(range(model.coef_.shape[1])), model.coef_[0]))
sorted_coef = sorted(zipped_coef, key= lambda v: v[1])
sorted_coef = np.array(sorted_coef)
sorted_coef_indexes = sorted_coef[:, 0].astype(int)
words = np.array(vectorizer.get_feature_names_out())
most_positive_words = words[sorted_coef_indexes[-10:]]
most_negative_words = words[sorted_coef_indexes[:10]]
print("most positive words: ", most_positive_words)
print("most negative words: ", most_negative_words)

#hint: model.coef_, vectorizer.get_feature_names()

most positive words:  ['worry' 'sooner' 'con' 'wonderfully' 'downside' 'saves' 'minor'
 'skeptical' 'lifesaver' 'penny']
most negative words:  ['dissapointed' 'worthless' 'theory' 'unusable' 'nope' 'worst' 'poorly'
 'disappointing' 'listened' 'shame']



Having previously partitioned the data, we now proceed to fit it to the suitable regression curve known as logistic regression. This enables us to instruct the program on discerning negative and positive words based on their occurrences in the numerical table. The computer is trained to associate a rating of -1 with more negative expressions in the reviews, saving this information for future recognition.

## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [44]:
#a)
model.predict(x_test)

array([ 1, -1,  1, ...,  1, -1,  1], dtype=int64)

In [45]:
#b)
probabilty = model.predict_proba(x_test)
probabilty

#hint: model.predict_proba()

array([[1.12343292e-03, 9.98876567e-01],
       [6.45599703e-01, 3.54400297e-01],
       [2.13849825e-04, 9.99786150e-01],
       ...,
       [6.31209931e-03, 9.93687901e-01],
       [9.99999657e-01, 3.43307809e-07],
       [4.50613157e-03, 9.95493868e-01]])

In [46]:
#c)
print("most positive: ", test.iloc[probabilty.T[1].argmax()]["review"])
print("\n most negative: ", test.iloc[probabilty.T[0].argmax()]["review"])

#hint: use the results of b)

most positive:  I bought this carrier when my daughter was about 4 weeks old shes now 10 weeks old  I had a Moby that I borrowed from a friend but could never quite get to work and my daughter hated being in it  I also have a Bjorn Active but she seemed pretty precarious in that when she was so littleThis carrier is nearly perfect for infants  Its not quite as easy to put on as the Bjorn but MUCH easier than the Moby and it gives that nice snug fit that the Moby did  Its much lighter weight than the Bjorn so its easier on my back  I have the khakicolored version so I havent had any problems with it showing dirt and dust  I wish the straps werent so long one size fits all but I just wrap them around to my back and tie them again loosely  Im 54 and 145 lbs  I think this carrier would fit most people  The fabric is thick and the construction seems to be of good quality It might be hot in hot weather but most carriers areThis has been a life saver  My daughter wont sleep anywhere except wh

In [47]:
#d) 
model.score(x_test, y_test)

0.9303769002428712

Building upon the actions undertaken in task 4, we proceed to identify the opinion containing the highest number of positive words and, conversely, the one with the most negative words. Ultimately, we assess the accuracy of our model, and the results indicate a high level of effectiveness, reaching an accuracy rate of 93%.

## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [53]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [54]:
#a)
vectorizer_small = CountVectorizer(vocabulary=significant_words)

x_small = vectorizer_small.fit_transform(list(train["review"]))
y_small = train["rating"]
x_test_small = vectorizer_small.transform(list(test["review"]))
y_test_small = test["rating"]
light_model = LogisticRegression()
light_model.fit(x_small, y_small)
proba = light_model.predict_proba(x_test_small)
print("most positive: ", test.iloc[proba.T[1].argmax()]["review"])
print("\nmost negative: ", test.iloc[proba.T[0].argmax()]["review"])
light_model.score(x_test_small, y_test_small)

most positive:  Check and recheck the KTan for size issues before letting your guard down and wearing NO ISSUES OCCURED however I thought I was good to go and then realized that this large was different than the large I had been given before it Odd but fabric This makes the moby wrap appear like a midevil torture devise and bonus you wont have to worry about some BabyWearing freak watching you in the parking lot at the mall as you struggle to turn a bolt of fabric into what the KTan has simplifiedseriously Dad approved and if the moby is your thing thats cool just perhaps realize that its not as easy as everyone thinks to wrap that amount of fabric around yourself evenly snug comfortable and without mummifying yourself and lord help you your baby  Another bonus to the KTAN issue with baby takes only a second to move her out of the KTAN but in a moby We thought for the first two days after returning the moby 34have you seen Katy34love it love it love it love it love it love it love it l

0.8675302089892357

In [50]:
#b)
for word, coef in zip(vectorizer_small.get_feature_names_out(), light_model.coef_[0]):
    print(f"{word}: {coef}")


love: 1.3989926093455205
great: 0.956031493402818
easy: 1.1842153716159887
old: 0.08116863112747041
little: 0.500933979332243
perfect: 1.4838465890012005
loves: 1.7282504597783226
well: 0.5018156386229154
able: 0.20141386562319
car: 0.07325059002318328
broke: -1.6816034616166022
less: -0.21324781686452343
even: -0.5186847771872509
waste: -1.9527997877757763
disappointed: -2.3661271204599887
work: -0.6450621082926463
product: -0.31786006654357496
money: -0.9293136885357741
would: -0.3471683883756012
return: -2.0392974055141204



Once more, we instruct our program to identify relevant words, but this time we employ a specific dictionary that we have provided. Given that the number of words in the list is less than the total count, we anticipate a potential decrease in prediction accuracy. Additionally, we evaluate the strength or weight assigned to each word at the conclusion of the process.

In [55]:
#c)
print(f"Light model - {light_model.score(x_test_small, y_test_small)}")
print(f"Standard model - {model.score(x_test, y_test)}")

#hint: %time, %timeit

Light model - 0.8675302089892357
Standard model - 0.9303769002428712


In [58]:
%%time
%%timeit
light_model.predict(x_test_small)

985 µs ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CPU times: total: 3.28 s
Wall time: 8.62 s


In [59]:
%%time
%%timeit
model.predict(x_test)

8.54 ms ± 710 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
CPU times: total: 3.16 s
Wall time: 7.73 s


Comparing our light model with original model we can observe that original model has around 6 % points better accuracy but with the cost of being around 9 times slower comparing to lighter model.